In [23]:
import cv2
import math
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.dataset import Subset
from sklearn.model_selection import KFold
import pandas as pd

from PIL import Image
import numpy as np
# from gradcam.utils import visualize_cam
import random

def torch_fix_seed(seed=0):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True


torch_fix_seed()

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# from vessel_MAE_new import vessel_MAE_new
# from vessel_MAE_mask50 import vessel_MAE_new
from vessel_models.vessel_MAE_TMD import vessel_MAE

In [24]:
data_path="/root/work/data/2021"

In [25]:
transform = transforms.Resize((48,512))

x_data=[]

for i in range(1,302):
    img=np.load(file=data_path+"/deck_depth_map/data/hull"+str(i)+".npy")
    img=torch.tensor(img).reshape(1,48,521)

    img=transform(img)

    # img.numpy()
    img=np.array(img[0])
    x_data.append(img)

x_data=np.array(x_data)
x_data = np.delete(x_data, range(117, 119), axis=0)
x_data=np.flip(x_data,axis=2)
x_data=np.array(x_data).reshape(299,1,48,512)
x_data=torch.tensor(x_data)

X_MAX = x_data.max()
x_data /= X_MAX

y =np.genfromtxt(data_path + "/hist_naked.dat", dtype=float, skip_header=1)
y = np.delete(y, range(117, 119), axis=0)

y_data=y[:,4:5]

In [26]:
class HullDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, index):
        return self.data[index], self.targets[index]

In [27]:
batch_size=60

data=torch.tensor(x_data).float().clone().detach()
targets=torch.tensor(y_data).float().clone().detach()

dataset=HullDataset(data,targets)

kfold=KFold(n_splits=5,shuffle=True,random_state=1)

train_loaders = []
val_loaders = []

val_index_all=[]

for train_index, val_index in kfold.split(dataset):  
    train_dataset = Subset(dataset, train_index)
    val_dataset = Subset(dataset, val_index)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    train_loaders.append(train_loader)
    val_loaders.append(val_loader)
    val_index_all.append(val_index)

/tmp/ipykernel_476752/2251032999.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data=torch.tensor(x_data).float().clone().detach()


In [28]:
result_path="/root/work/result/RMSPE/TMD/"

In [29]:
# plt.rcParams["figure.figsize"] = (16, 2)

# df = pd.DataFrame(columns=['Index', 'factor Loss', 'MIM Loss'])
rmspe_loss=[]

for i in range(5):
    loss=0
    fold_num = i
    val_loader = val_loaders[fold_num]
    val_indexs = val_index_all[fold_num]


    model=vessel_MAE()
    pth_file_path = result_path+"model"+str(i)+".pth"
    model.load_state_dict(torch.load(pth_file_path))
    model.to(device)
    model.eval()


    for inputs_test, labels_test in val_loader:
        inputs_test = inputs_test.to(device)
        labels_test = labels_test.to(device)
        factor_loss, mim_loss, factor = model(inputs_test,labels_test)

    for i in range(len(factor)):
        loss+=((labels_test[i].item()-factor[i].item())/labels_test[i].item())**2

    loss=math.sqrt(loss/len(factor))
    rmspe_loss.append(loss)

    # print(factor)
    # print(labels_test)




# df = df.sort_values(by='Index')
# df.to_csv(result_path+'loss.csv', index=False)


In [35]:
print(rmspe_loss)
print(np.mean(rmspe_loss)*100)

[0.020206510588713825, 0.01920228016214507, 0.028224977643752223, 0.011046824134444785, 0.01601101772674013]
1.8938322051159207
